In [2]:
import cv2
import pandas as pd
from tqdm import tqdm
from glob import glob, os
from brisque import BRISQUE
from matplotlib import pyplot as plt
from IPython.display import clear_output

search_dir = 'D:\\Projects\\Ship_Campaign\\ShipRSImageNet_V1\\ShipRSImageNet_V1\\VOC_Format\\JPEGImages\\'
quality_list = []

def image_quality(filename):
    qlty = BRISQUE(search_dir + filename)
    scr = round(qlty.score(), 2)
    return scr

In [ ]:
# This Takes More Time
for img in tqdm(glob(search_dir + '*.bmp'), desc='Getting Quality Of Images'):
    img_name = os.path.basename(img)
    img_qty = image_quality(img_name)
    quality_list.append((img_name, img_qty))
print('Completed !!!')

Getting Quality Of Images: 100%|██████████| 3436/3436 [1:12:25<00:00,  1.26s/it]

Completed !!!


In [ ]:
# Rename File Name Befor Use
qty_df = pd.DataFrame(quality_list, columns=['image_name', 'quality'])
qty_df = qty_df.sort_values(by='quality', ignore_index=True)
qty_df.to_csv(search_dir + 'ShipRSImageNet_V1.csv', index=False)

In [117]:
import pandas as pd
qcsv = pd.read_csv(search_dir + 'ShipRSImageNet_V1.csv')
qcsv.describe()

,quality
count,3436.000000
mean,32.063036
std,16.715327
min,-24.780000
25%,20.750000
50%,31.530000
75%,43.555000
max,119.810000


In [118]:
maxv = qcsv.quality.describe()['max']
half = qcsv.quality.describe()['25%']
img_check = qcsv[qcsv.quality <= 10].reset_index(drop=True)
display(img_check)

,image_name,quality
0,100000951.bmp,-24.78
1,000755.bmp,-23.53
2,000838.bmp,-21.11
3,100001173.bmp,-15.39
4,100001243.bmp,-15.29
...,...,...
285,100001002.bmp,9.86
286,100001254.bmp,9.90
287,000625.bmp,9.93
288,000814.bmp,9.99


In [ ]:
qualified_frames = []
for index, row in img_check.iterrows():
    clear_output(wait=True)
    iname = row.image_name
    qty = row.quality
    image = cv2.imread(search_dir + iname)
    plt.figure(figsize=(15,15))
    plt.axis('off')
    plt.imshow(image)
    plt.title(f'{iname} : {qty}')
    qc = input('Qualified?')
    if qc == 1:
        qualified_frames.append((iname, qty))
    elif qc == 0:
        pass
    else: break

<h3>Cropping and Categorizing Ship Images for Instruction Document</h3>

In [119]:
import json
from PIL import Image
from ast import literal_eval
import matplotlib.pyplot as plt

anno_df = pd.read_csv('Level_2.csv', converters={'bbox' : eval, 'segmentation' : eval})
anno_df.name = anno_df.name.astype(str)
print(len(anno_df))
print(len(anno_df.file_name.unique()))

10860
2198


In [120]:
anno_df = pd.merge(anno_df, img_check, left_on='file_name', right_on='image_name').reset_index(drop=True)
anno_df

,area,category_id,ignore,segmentation,iscrowd,bbox,image_id,id,name,supercategory,file_name,width,height,image_name,quality
0,2800.0,15,0,"[[182.0, 178.0, 199.0, 161.0, 120.0, 82.0, 103...",0,"[103, 82, 97, 97]",82,285,Cargo,Cargo,004284.bmp,930,930,004284.bmp,5.50
1,3472.0,15,0,"[[304.0, 276.0, 324.0, 258.0, 242.0, 166.0, 22...",0,"[222, 166, 103, 111]",82,286,Cargo,Cargo,004284.bmp,930,930,004284.bmp,5.50
2,23069.0,9,0,"[[529.0, 360.0, 284.0, 56.0, 239.0, 92.0, 484....",0,"[239, 56, 291, 341]",82,289,Landing,Landing,004284.bmp,930,930,004284.bmp,5.50
3,3348.0,13,0,"[[304.0, 275.0, 324.0, 258.0, 242.0, 167.0, 22...",0,"[222, 167, 103, 109]",82,287,Container Ship,Container Ship,004284.bmp,930,930,004284.bmp,5.50
4,10528.0,13,0,"[[372.0, 751.0, 410.0, 724.0, 280.0, 543.0, 24...",0,"[242, 543, 169, 209]",82,288,Container Ship,Container Ship,004284.bmp,930,930,004284.bmp,5.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,2047.0,18,0,"[[868.0, 254.0, 886.0, 242.0, 840.0, 167.0, 82...",0,"[822, 167, 65, 88]",697,3581,Ferry,Ferry,1120__920_1840.bmp,930,930,1120__920_1840.bmp,-11.78
846,3014.0,18,0,"[[521.0, 411.0, 516.0, 391.0, 385.0, 427.0, 39...",0,"[385, 391, 137, 57]",902,4414,Ferry,Ferry,000391.bmp,930,930,000391.bmp,3.42
847,39032.0,18,0,"[[708.0, 615.0, 356.0, 297.0, 302.0, 357.0, 65...",0,"[302, 297, 407, 379]",1923,9517,Ferry,Ferry,000292.bmp,930,930,000292.bmp,-1.07
848,12985.0,5,0,"[[555.0, 650.0, 585.0, 666.0, 760.0, 341.0, 73...",0,"[555, 325, 206, 342]",1100,5299,Cruiser,Cruiser,100001002.bmp,1166,808,100001002.bmp,9.86


In [114]:
img_dir = rf'D:/Projects/Ship_Campaign/ShipRSImageNet_V1/ShipRSImageNet_V1/VOC_Format/JPEGImages/'

for cat in tqdm(anno_df.name.unique(), desc="Cropping Images"):
    cat_df = anno_df[anno_df.name == cat].reset_index(drop=True)
    for index, row in cat_df.iterrows():
        fimg = img_dir + row.file_name
        cat_name = row.supercategory
        file_name = row.file_name
        bbox = row.bbox
        bbox = [bbox[0]+10,  bbox[1]+10, bbox[0]+bbox[2]+10, bbox[1]+bbox[3]+10]
        img = Image.open(fimg)
        img = img.crop(bbox)
        img.save(f'cropped_categories\{cat_name}_{index}_({file_name}).png')

Cropping Images: 100%|██████████| 25/25 [12:40<00:00, 30.42s/it]


<h3>Getting the Orignal Image and Renamed Image of TOD for Kapil Metrix Derivation</h3>

In [97]:
import json
import pandas as pd
import numpy as np
import glob
import cv2
import imagehash as imh
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

coco_name = 'TOD/Org_Coco_Name.csv'
coco_name_df = pd.read_csv(coco_name)
chk_img_path = 'D:/Projects/TOD/tod_campaign_v1/All_Images/'

org_file_names = []
cr_file_names = []

for index, row in tqdm(coco_name_df.iterrows(), total=coco_name_df.shape[0], desc="Processing Original Images"):
    org_name = row.Org_Coco_Name
    hash_value = imh.dhash(Image.open(row.Path))
    org_file_names.append((org_name, hash_value))

for c_img_file in tqdm(glob.glob(chk_img_path + '*.*'), desc="Processing Current Images"):
    cr_name = glob.os.path.basename(c_img_file)
    hash_value = imh.dhash(Image.open(c_img_file))
    cr_file_names.append((cr_name, hash_value))

Processing Current Images: 100%|██████████| 1050/1050 [01:43<00:00, 10.13it/s]


In [ ]:
org_df = pd.DataFrame(org_file_names, columns=['image','hash_value'])
cr_df = pd.DataFrame(cr_file_names, columns=['image','hash_value'])
final_df = pd.merge(org_df, cr_df, on='hash_value')
final_df = final_df.rename(columns={'image_x':'original_name', 'image_y':'current_name'})
final_df.original_name = final_df.original_name.str.replace('\\', '/', regex=True)
final_df.to_csv('matched_images.csv', index=False)

In [75]:
final_df = pd.read_csv('matched_images.csv')
print(len(final_df))
map_dict = final_df.set_index('original_name').T.to_dict()
print(len(map_dict))

1043
1043


In [ ]:
coco_file = "D:/Tasks/prototypes/TOD/archive/data/annotations.json"
with open(coco_file, 'r') as file:
    coco_json = json.loads(file.read())
    print(coco_json.keys())
    file.close()

for img_data in tqdm(coco_json['images'], desc="Changing Original COCO to New COCO"):
    coco_img_name = img_data['file_name']
    try:
        new_name = map_dict[coco_img_name]['current_name']
        img_data['file_name'] = new_name
    except KeyError:
        pass

with open('new_coco.json', 'w') as new_file:
    w_json = json.dump(coco_json, new_file)
    new_file.close()

In [116]:
image_map = pd.read_csv('image_map.csv', converters={'bbox': eval, 'segmentation': eval})
image_map.dtypes

id                  int64
image_id            int64
category_id         int64
segmentation       object
area              float64
bbox               object
iscrowd             int64
supercategory      object
name               object
width               int64
height              int64
file_name          object
license            object
flickr_url         object
coco_url          float64
date_captured     float64
flickr_640_url     object
dtype: object